In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Supress auto scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Load Datasets

### Top airbnb zipcodes throughout nyc  (2015-2022)
Get top airbnb zip codes that need to be merged with zillow home values

In [3]:
top_zipcodes = pd.read_csv("./zipcode_data/unmapped_data/nyc_top_zipcodes_count.csv")
display(top_zipcodes)

,zipcode,borough,year,count
0,10469,BRONX,2022,122
1,10466,BRONX,2022,119
2,10463,BRONX,2018,105
3,10466,BRONX,2021,99
4,10456,BRONX,2022,98
5,11211,BROOKLYN,2019,6231
6,11211,BROOKLYN,2018,3227
7,11211,BROOKLYN,2022,2730
8,11211,BROOKLYN,2020,2719
9,11211,BROOKLYN,2017,2645


In [4]:
# EXPORT
# top_zipcodes.to_csv("./zipcode_data/nyc_top_zipcodes_all_years.csv", index=False)
# display(top_zipcodes)
# print("CSV Generated")

### Rent values (2002-2022)
Get rent values for each zipcode in nyc

In [5]:
rent_vals = pd.read_csv('../data/clean/nyc_rental_prices.csv', low_memory=False)
display(rent_vals)

,rent,borough,zipcode,year
0,16000.000,MANHATTAN,10013,2022
1,8250.000,MANHATTAN,10012,2022
2,2200.000,MANHATTAN,10031,2022
3,13000.000,MANHATTAN,10011,2022
4,55000.000,MANHATTAN,10065,2022
...,...,...,...,...
9070,7836.500,MANHATTAN,10282,2022
9071,5778.600,MANHATTAN,10069,2022
9072,4823.200,MANHATTAN,10004,2022
9073,4180.500,MANHATTAN,10006,2022


In [6]:
# EXPORT
# rent_vals.to_csv("../data/clean/nyc_rental_prices.csv", index=False)
# display(rent_vals)
# print("CSV Generated")

# Filter by significant years and compute yearly mean

### Filter by significant years from top zipcodes
Find min and max year for top zipcodes that need to be mapped from "zillow_home_values"

In [7]:
# min_year, max_year = top_zipcodes["year"].min(), top_zipcodes["year"].max()
# print("MIN={}\nMAX={}".format(min_year, max_year))

Use **MIN, MAX** years from top_zipcodes and narrow down zillow home_value years between [min, max]

In [8]:
# NOTE: Already simplified, no change...
# rent_vals = rent_vals[rent_vals["year"].between(min_year, max_year, inclusive="both")]
# print("Years present in Airbnb zipcode rent value:", rent_vals["year"].unique().tolist(), end="\n\n")
# rent_vals.info()

### Compute yearly mean
Group by **zipcode, year, and borough** to compute the average for home_value for each zipcode on a yearly basis. Entry amount stays the same.

In [9]:
rent_vals = rent_vals.groupby(["zipcode", "year", "borough"])["rent"].mean().to_frame().reset_index()
rent_vals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   zipcode  867 non-null    int64  
 1   year     867 non-null    int64  
 2   borough  867 non-null    object 
 3   rent     867 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.2+ KB


# Merge airbnb top zipcode count count to zillow rent values
Merge all the top 5 zipcodes for each borough to their appropriate rent values based on
- **year, zipcode, and borough**

### Dataframes

In [10]:
print("top_zipcodes")
display(top_zipcodes.head())
print("rent_vals")
display(rent_vals.head())

top_zipcodes


,zipcode,borough,year,count
0,10469,BRONX,2022,122
1,10466,BRONX,2022,119
2,10463,BRONX,2018,105
3,10466,BRONX,2021,99
4,10456,BRONX,2022,98


rent_vals


,zipcode,year,borough,rent
0,7020,2022,QUEENS,2741.000
1,10001,2015,MANHATTAN,4010.000
2,10001,2016,MANHATTAN,4022.983
3,10001,2017,MANHATTAN,3958.383
4,10001,2018,MANHATTAN,4001.067


### Merging dataframes

In [11]:
merge_cols = ["year", "zipcode", "borough"]
merged_top_zipcodes = pd.merge(top_zipcodes, rent_vals,  how='left', left_on=merge_cols, right_on=merge_cols)
print("merged_top_zipcodes")
display(merged_top_zipcodes)

merged_top_zipcodes


,zipcode,borough,year,count,rent
0,10469,BRONX,2022,122,1883.011
1,10466,BRONX,2022,119,1946.714
2,10463,BRONX,2018,105,2308.025
3,10466,BRONX,2021,99,NaN
4,10456,BRONX,2022,98,2152.938
5,11211,BROOKLYN,2019,6231,3335.467
6,11211,BROOKLYN,2018,3227,3189.117
7,11211,BROOKLYN,2022,2730,4141.138
8,11211,BROOKLYN,2020,2719,3324.808
9,11211,BROOKLYN,2017,2645,3226.850


In [12]:
# Generate Shapiro wilk test for 
def generate_shapiro_test(df, boro):
    data = df[df["borough"] == boro]
    shapiro_wilk_data = stats.shapiro(data['rent'])
    return "{}:::W={}, P={}".format(boro, shapiro_wilk_data[0], shapiro_wilk_data[1])

### Test for normality

In [13]:
boros = top_zipcodes["borough"].unique()
[generate_shapiro_test(merged_top_zipcodes, boro) for boro in boros]

['BRONX:::W=nan, P=1.0',
 'BROOKLYN:::W=0.6957744359970093, P=0.008645490743219852',
 'MANHATTAN:::W=0.7729074358940125, P=0.047846902161836624',
 'QUEENS:::W=nan, P=1.0',
 'STATEN_ISLAND:::W=nan, P=1.0']

# Merge remaining airbnb zipcode count  to zillow home values
Merge remaining zillow zipcodes (excluding top 5) to remaining airbnb count based on
- **year, zipcode, and borough**

### Dataframes

In [14]:
remaining_zipcode_count = pd.read_csv("./zipcode_data/unmapped_data/nyc_remaining_zipcode_count.csv")
print("remaining_zipcode_count")
display(remaining_zipcode_count.head())
print("rent_vals")
display(rent_vals.head())

remaining_zipcode_count


,zipcode,borough,year,count
0,10001,MANHATTAN,2015,350
1,10001,MANHATTAN,2016,666
2,10001,MANHATTAN,2017,418
3,10001,MANHATTAN,2018,491
4,10001,MANHATTAN,2019,275


rent_vals


,zipcode,year,borough,rent
0,7020,2022,QUEENS,2741.000
1,10001,2015,MANHATTAN,4010.000
2,10001,2016,MANHATTAN,4022.983
3,10001,2017,MANHATTAN,3958.383
4,10001,2018,MANHATTAN,4001.067


### Merging dataframes

In [15]:
merge_cols = ["year", "zipcode", "borough"]
merged_reminaing = remaining_zipcode_count.merge(rent_vals, indicator=True, how="left", left_on=merge_cols, right_on=merge_cols).drop(columns=['_merge'])
print("merged_reminaing")
display(merged_reminaing.head())
print("count:", len(merged_reminaing))

merged_reminaing


,zipcode,borough,year,count,rent
0,10001,MANHATTAN,2015,350,4010.000
1,10001,MANHATTAN,2016,666,4022.983
2,10001,MANHATTAN,2017,418,3958.383
3,10001,MANHATTAN,2018,491,4001.067
4,10001,MANHATTAN,2019,275,4145.267


count: 1404


In [17]:
# EXPORT
merged_reminaing.to_csv("./zipcode_data/mapped_data/remaining_zipcodes_rent_val_mapped.csv", index=False)
display(merged_reminaing)
print("CSV Generated")

,zipcode,borough,year,count,rent
0,10001,MANHATTAN,2015,350,4010.000
1,10001,MANHATTAN,2016,666,4022.983
2,10001,MANHATTAN,2017,418,3958.383
3,10001,MANHATTAN,2018,491,4001.067
4,10001,MANHATTAN,2019,275,4145.267
...,...,...,...,...,...
1399,11694,QUEENS,2020,11,NaN
1400,11694,QUEENS,2021,43,NaN
1401,11694,QUEENS,2022,64,3133.333
1402,11697,QUEENS,2016,1,NaN


CSV Generated


# Check for missing home values from remaining zipcodes

In [ ]:
#Rockerfeller center 10020, check what type of areas we're missing.Look for significant counts andor/ places
missing_zipcodes = merged_reminaing[merged_reminaing["rent"].isnull()]   

In [ ]:
missing_zipcodes.to_csv("./zipcode_data/missing_zipcode_rent_values.csv", index=False)
display(missing_zipcodes)
print("CSV Generated")

In [ ]:
def get_boro_missing_zipcodes(df, boro):
    missing_boro = df[df["borough"] == boro]
    missing_years = missing_boro["year"].unique().tolist()
    missing_zipcodes = missing_boro["zipcode"].unique().tolist()
    missing_zipcodes.sort();     missing_years.sort()
    str1 = "{}\nmissing zipcode_count:{}".format(boro, len(missing_boro))
    str2 = "\nmissing years: {}".format(missing_years)
    str3 = "\nmissing zipcodes:{}\n\n".format(missing_zipcodes)
    return str1 + str2 + str3

### Explain in EDA as to why these zipcodes are missing home values and not necessary
- So far, **MANHATTAN** has justifiable zipcodes along with parts of **BROOKLYN**
- TODO: **BRONX**, **QUEENS**

In [ ]:
boros = top_zipcodes["borough"].unique()
[print(get_boro_missing_zipcodes(missing_zipcodes, boro)) for boro in boros]; display()